In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [72]:
Labeled_Metadata = pd.read_csv('C:\\Users\\Steffi Grace\\24592774_LLM_ILabResearch\\Notebooks\\Datasets\\Labeled_Metadata.csv')

In [73]:
Labeled_Metadata.count()

query                1273
title                1273
description          1273
summary              1273
tags                 1001
dataset_url          1273
available_formats    1273
label                1273
dtype: int64

In [68]:
Labeled_Metadata.head()

,query,title,description,summary,tags,dataset_url,available_formats,label
0,electric vehicle,2023w7 global electric vehicle market share,global electric vehicle market share,global electric vehicle market share,makeover monday cars vehicles electric ev mark...,https://data.world/makeovermonday/2023w7,['xlsx'],1
1,electric vehicle,connected electric autonomous vehicle,connected electric autonomous vehicle,romanian new car registration in 2023 vs previ...,autonomous,https://data.world/smartcolumbusos/650b7e59-af...,['csv'],1
2,electric vehicle,electric vehicle charging stations,electric vehicle charging stations,statewise data from 2001 is classified accordi...,electric vehicle environment energy,https://data.world/townofcary/electric-vehicle...,"['dbf', 'prj', 'json', 'shp', 'csv', 'shx']",1
3,electric vehicle,nyserda electric vehicle drive clean rebate da...,new york states charge ny initiative offers el...,coronoavirus covid19 data updated daily,ev electric vehicle bev phev ghg drive clean r...,https://data.world/data-ny-gov/thd2-fu8y,['csv'],1
4,electric vehicle,impact of uncoordinated plugin electric vehicl...,impact of uncoordinated plugin electric vehicl...,adding public datasets related to corona virus...,battery consumption data energy hybrid midwest...,https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx'],1


In [56]:
# Filter the labeled metadata with label == 1
filtered_metadata = Labeled_Metadata[Labeled_Metadata['label'] == 1]

Labeled_Metadata = filtered_metadata.copy()

In [57]:
Labeled_Metadata.count()

query                76
title                76
description          76
summary              76
tags                 61
dataset_url          76
available_formats    76
label                76
dtype: int64

In [58]:
Labeled_Metadata.head()

,query,title,description,summary,tags,dataset_url,available_formats,label
0,electric vehicle,2023w7 global electric vehicle market share,global electric vehicle market share,global electric vehicle market share,makeover monday cars vehicles electric ev mark...,https://data.world/makeovermonday/2023w7,['xlsx'],1
1,electric vehicle,connected electric autonomous vehicle,connected electric autonomous vehicle,romanian new car registration in 2023 vs previ...,autonomous,https://data.world/smartcolumbusos/650b7e59-af...,['csv'],1
2,electric vehicle,electric vehicle charging stations,electric vehicle charging stations,statewise data from 2001 is classified accordi...,electric vehicle environment energy,https://data.world/townofcary/electric-vehicle...,"['dbf', 'prj', 'json', 'shp', 'csv', 'shx']",1
3,electric vehicle,nyserda electric vehicle drive clean rebate da...,new york states charge ny initiative offers el...,coronoavirus covid19 data updated daily,ev electric vehicle bev phev ghg drive clean r...,https://data.world/data-ny-gov/thd2-fu8y,['csv'],1
4,electric vehicle,impact of uncoordinated plugin electric vehicl...,impact of uncoordinated plugin electric vehicl...,adding public datasets related to corona virus...,battery consumption data energy hybrid midwest...,https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx'],1


In [69]:
metadata = Labeled_Metadata.copy()

In [70]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from tqdm import tqdm

# Define Dataset Class
class QueryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        query = row['query']
        title = row['title']
        description = row['description']
        summary = row['summary']
        label = row['label']
        inputs = self.tokenizer.encode_plus(
            query,
            title + ' ' + description + ' ' + summary,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

metadata = metadata.copy()

# Define max_len
max_len = 128

# Prepare Dataset and DataLoader
train_dataset = QueryDataset(metadata, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define Optimizer and Loss Function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
model.train()
for epoch in range(3):  
    print(f"Epoch {epoch+1}")
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Average Training Loss: {total_loss / len(train_loader)}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1


  0%|          | 0/160 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 1/160 [00:02<06:41,  2.52s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, 

Average Training Loss: 0.23898677096003668
Epoch 2


  0%|          | 0/160 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 1/160 [00:02<05:46,  2.18s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, 

Average Training Loss: 0.15249908713740296
Epoch 3


  0%|          | 0/160 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|▏         | 2/160 [00:04<05:44,  2.18s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, 

Average Training Loss: 0.08725945386395324


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Relevant dataset URLs sorted by similarity score (descending order):
Dataset URL: https://data.world/rajanand/crime-in-india
Title: crime in india
Description: statewise data from 2001 is classified according to 40factors 75 csv files
Summary: statewise data from 2001 is classified according to 40factors 75 csv files
Score: 0.8970213532447815


Dataset URL: https://data.world/rajanand/crime-in-india
Title: crime in india
Description: statewise data from 2001 is classified according to 40factors 75 csv files
Summary: statewise data from 2001 is classified according to 40factors 75 csv files
Score: 0.8970213532447815


Dataset URL: https://data.world/rajanand/crime-in-india
Title: crime in india
Description: statewise data from 2001 is classified according to 40factors 75 csv files
Summary: statewise data from 2001 is classified according to 40factors 75 csv files
Score: 0.8970213532447815


Dataset URL: https://data.world/rajanand/crime-in-india
Title: crime in india
Description: statew

In [78]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from nltk import word_tokenize,WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

metadata = pd.read_csv('C:\\Users\\Steffi Grace\\24592774_LLM_ILabResearch\\Notebooks\\Datasets\\Labeled_Metadata.csv')

def preprocess_text(text):
    if text == 'nan':
        return ""
    if isinstance(text, str):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens if token.isalpha() and len(token) > 1]
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    else:
        return ""

for column in ['title', 'description', 'summary']:
    metadata[column] = metadata[column].apply(preprocess_text)

metadata_embeddings = []
for index, row in metadata.iterrows():
    text = row['title'] + ' ' + row['description'] + ' ' + row['summary']
    tokenized_text = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokenized_text)
    logits = output.logits  
    embeddings = logits.mean(dim=1).squeeze().numpy() 
    metadata_embeddings.append(embeddings)


user_input = input("Enter your query: ")
tokenized_input = tokenizer(user_input, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    output = model(**tokenized_input)
user_embeddings = output.logits.mean(dim=1).squeeze().numpy()


similarities = cosine_similarity([user_embeddings], metadata_embeddings)[0]

def adjust_score(score, title, description, summary):
    completeness_factor = 0
    if title:
        completeness_factor += 0.1
    if description:
        completeness_factor += 0.1
    if summary:
        completeness_factor += 0.1
    return score + completeness_factor

adjusted_similarities = []
for i, score in enumerate(similarities):
    title, description, summary = metadata.iloc[i]['title'], metadata.iloc[i]['description'], metadata.iloc[i]['summary']
    adjusted_score = adjust_score(score, title, description, summary)
    adjusted_similarities.append(adjusted_score)

# Calculate the 95th percentile threshold
percentile_threshold = np.percentile(adjusted_similarities, 98)

dataset_urls_with_scores = zip(metadata['dataset_url'], adjusted_similarities)
sorted_dataset_urls_with_scores = sorted(dataset_urls_with_scores, key=lambda x: x[1], reverse=True)

print("Relevant dataset URLs sorted by similarity score (descending order):")
for counter, (dataset_url, similarity_score) in enumerate(sorted_dataset_urls_with_scores, start=1):
    if similarity_score >= percentile_threshold:
        print(f"Counter: {counter}")
        print(f"Similarity Score: {similarity_score}")
        print(f"Dataset URL: {dataset_url}")
        metadata_index = metadata.index[metadata['dataset_url'] == dataset_url].tolist()[0]
        print(f"Title: {metadata['title'].iloc[metadata_index]}")
        print(f"Description: {metadata['description'].iloc[metadata_index]}")
        print(f"Summary: {metadata['summary'].iloc[metadata_index]}")
        print("\n")
        total_output = counter


ValueError: Expected 2D array, got 1D array instead:
array=[0.23985833].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(query, title, description, summary):
    # Concatenate the title, description, and summary into a single text
    text = title + ' ' + description + ' ' + summary

    # Use TF-IDF vectorizer to convert the text into a numerical vector representation
    vectorizer = TfidfVectorizer()
    text_vectors = vectorizer.fit_transform([query, text])

    # Calculate cosine similarity between the query vector and the text vector
    similarity_score = cosine_similarity(text_vectors)[0, 1]

    return similarity_score


In [75]:
# Given a user query:
user_query = "electric vehicle charging stations"

# Use the fine-tuned BERT model to predict relevance scores for each dataset
relevant_datasets = []
for index, row in Labeled_Metadata.iterrows():
    relevance_score = model.predict_relevance(user_query, row['query'])  # Use the fine-tuned model to predict relevance
    if relevance_score >= 0.95:  
        dataset_title = row['title']
        dataset_description = row['description']
        dataset_summary = row['summary']
        similarity_score = calculate_similarity(user_query, dataset_title, dataset_description, dataset_summary)
        relevant_datasets.append((row['dataset_url'], similarity_score))

# Sort relevant datasets by similarity score and recommend the top ones
relevant_datasets = sorted(relevant_datasets, key=lambda x: x[1], reverse=True)
top_relevant_datasets = relevant_datasets[:5]  

AttributeError: 'BertForSequenceClassification' object has no attribute 'predict_relevance'

In [71]:
def predict_relevant_datasets(user_query, tokenizer, model, dataframe, max_len=128):
    model.eval()
    query_dataset = QueryDataset(dataframe, tokenizer, max_len)
    data_loader = DataLoader(query_dataset, batch_size=1, shuffle=False)

    scores = []
    for batch in data_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        relevant_probability = probabilities[:, 1]  # Probability of being relevant
        scores.extend(relevant_probability.tolist())

    dataframe['score'] = scores
    sorted_df = dataframe[dataframe['score'] >= 0.98]  # Filter by cosine similarity threshold
    sorted_df = sorted_df.sort_values(by='score', ascending=False)

    return sorted_df

metadata = metadata.copy()

# User input for prediction
user_input = input("Enter your query: ")

# Prepare the dataframe for prediction
predict_df = pd.DataFrame({
    'query': [user_input] * len(metadata),
    'title': metadata['title'],
    'description': metadata['description'],
    'summary': metadata['summary'],
    'dataset_url': metadata['dataset_url'],
    'label': metadata['label']  
})

# Predict relevant datasets
predicted_df = predict_relevant_datasets(user_input, tokenizer, model, predict_df)

# Display the relevant dataset URLs sorted by similarity score
print("Relevant dataset URLs sorted by similarity score (descending order):")
for index, row in predicted_df.iterrows():
    print(f"Dataset URL: {row['dataset_url']}")
    print(f"Title: {row['title']}")
    print(f"Description: {row['description']}")
    print(f"Summary: {row['summary']}")
    print(f"Score: {row['score']}")
    print("\n")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Relevant dataset URLs sorted by similarity score (descending order):


In [59]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

class QueryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        query = row['query']
        text = row['title'] + ' ' + row['description'] + ' ' + row['summary']
        inputs = self.tokenizer.encode_plus(
            query,
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        label = torch.tensor(row['label'], dtype=torch.long)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Create the dataset and dataloaders
max_len = 128
train_dataset = QueryDataset(Labeled_Metadata, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10
)

In [62]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [63]:
# Fine-tune the model

trainer.train()

  0%|          | 0/30 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the sett

{'loss': 0.193, 'grad_norm': 0.37442293763160706, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


 37%|███▋      | 11/30 [00:21<00:33,  1.74s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 40%|████      | 12/30 [00:23<00:31,  1.76s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 4

{'loss': 0.0068, 'grad_norm': 0.08789453655481339, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}


 70%|███████   | 21/30 [00:39<00:15,  1.76s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 73%|███████▎  | 22/30 [00:41<00:14,  1.77s/it]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be

{'loss': 0.0027, 'grad_norm': 0.047431401908397675, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 56.3805, 'train_samples_per_second': 4.044, 'train_steps_per_second': 0.532, 'train_loss': 0.06751992963254452, 'epoch': 3.0}


TrainOutput(global_step=30, training_loss=0.06751992963254452, metrics={'train_runtime': 56.3805, 'train_samples_per_second': 4.044, 'train_steps_per_second': 0.532, 'train_loss': 0.06751992963254452, 'epoch': 3.0})

In [64]:
Labeled_Metadata.head()

,query,title,description,summary,tags,dataset_url,available_formats,label
0,electric vehicle,2023w7 global electric vehicle market share,global electric vehicle market share,global electric vehicle market share,makeover monday cars vehicles electric ev mark...,https://data.world/makeovermonday/2023w7,['xlsx'],1
1,electric vehicle,connected electric autonomous vehicle,connected electric autonomous vehicle,romanian new car registration in 2023 vs previ...,autonomous,https://data.world/smartcolumbusos/650b7e59-af...,['csv'],1
2,electric vehicle,electric vehicle charging stations,electric vehicle charging stations,statewise data from 2001 is classified accordi...,electric vehicle environment energy,https://data.world/townofcary/electric-vehicle...,"['dbf', 'prj', 'json', 'shp', 'csv', 'shx']",1
3,electric vehicle,nyserda electric vehicle drive clean rebate da...,new york states charge ny initiative offers el...,coronoavirus covid19 data updated daily,ev electric vehicle bev phev ghg drive clean r...,https://data.world/data-ny-gov/thd2-fu8y,['csv'],1
4,electric vehicle,impact of uncoordinated plugin electric vehicl...,impact of uncoordinated plugin electric vehicl...,adding public datasets related to corona virus...,battery consumption data energy hybrid midwest...,https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx'],1


In [65]:
# Save the model
model.save_pretrained('./trained_model_BERT_02')
tokenizer.save_pretrained('./trained_model_BERT_02')

('./trained_model_BERT_02\\tokenizer_config.json',
 './trained_model_BERT_02\\special_tokens_map.json',
 './trained_model_BERT_02\\vocab.txt',
 './trained_model_BERT_02\\added_tokens.json')

In [66]:
import torch.nn.functional as F

def predict_relevant_datasets(user_query, tokenizer, model, dataframe, max_len=128):
    model.eval()
    query_dataset = QueryDataset(dataframe, tokenizer, max_len)
    data_loader = DataLoader(query_dataset, batch_size=1, shuffle=False)

    scores = []
    for batch in data_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        positive_probability = probabilities[:, 1]  
        scores.extend(positive_probability.tolist())  

    dataframe['score'] = scores
    sorted_df = dataframe.sort_values(by='score', ascending=False)

    return sorted_df



# User input for prediction
user_input = input("Enter your query: ")

# Prepare the dataframe for prediction
predict_df = pd.DataFrame({
    'query': [user_input] * len(Labeled_Metadata),
    'title': Labeled_Metadata['title'],
    'description': Labeled_Metadata['description'],
    'summary': Labeled_Metadata['summary'],
    'dataset_url': Labeled_Metadata['dataset_url'],
    'label': Labeled_Metadata['label']
})

# Predict relevant datasets
predicted_df = predict_relevant_datasets(user_input, tokenizer, model, predict_df)

# Display the relevant dataset URLs sorted by similarity score
print("Relevant dataset URLs sorted by similarity score (descending order):")
for index, row in predicted_df.iterrows():
    print(f"Dataset URL: {row['dataset_url']}")
    print(f"Title: {row['title']}")
    print(f"Description: {row['description']}")
    print(f"Summary: {row['summary']}")
    print(f"Score: {row['score']}")
    print("\n")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Relevant dataset URLs sorted by similarity score (descending order):
Dataset URL: https://data.world/city-of-ny/njjd-3gve
Title: 20192020 arts survey data
Description: the annual arts education survey collects information on student participation in and access to arts education at
Summary: the annual arts education survey collects information on student participation in and access to arts education at nycdoe schools please note the following artsrelated data are now collected from other sources the number of certified art teachers and noncertified teachers teaching the arts is collected form the hr and beds survey the arts instructional hours provided to elementary students are collected from the student transcript and academic recording system stars the middle and high school participation in the arts data and the nysed requirement data are collected form stars and the hs arts sequence data are also collected form stars source httpsdatacityofnewyorkusdnjjd3gve last updated at httpsdat